In [1]:
import os
os.chdir('..')

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
df = pd.read_csv(".data/sales_train.csv")

In [ ]:
df.columns

In [ ]:
%timeit df.to_csv(".data/tests/df.csv", index=False)

In [ ]:
%timeit df.to_parquet(".data/tests/df_snappy.parquet", engine='pyarrow')

In [ ]:
%timeit df.to_parquet(".data/tests/df_gzip.parquet", engine='pyarrow', compression='gzip')

In [ ]:
%timeit df.to_parquet(".data/tests/df_brotli.parquet", engine='pyarrow', compression='brotli')

In [ ]:
%timeit df.to_feather(".data/tests/df.feather")

In [ ]:
!ls -lahS .data/tests

In [3]:
df.to_parquet(".data/tests/df_date_partition_parquet", partition_cols=["date"], engine='pyarrow')

NameError: name 'df' is not defined

In [ ]:
pd.read_parquet(".data/tests/df_date_partition_parquet/date=01.01.2013/f97c96c58e554423bfde4d486bb60eb0-0.parquet")

In [ ]:
df.to_parquet(".data/tests/df_date_block_num_partition_parquet", partition_cols=["date_block_num"], engine='pyarrow')

In [ ]:
pd.read_parquet(".data/tests/df_date_block_num_partition_parquet/date_block_num=0/25ea1d93c43e4d33808821a209ba3d53-0.parquet")

In [ ]:
!du -sh .data/tests/df_date_block_num_partition_parquet

In [ ]:
!du -sh .data/tests/df_date_partition_parquet

In [4]:
import deltalake
import pyarrow as pa

In [5]:
path_pq = ".data/df_agg_monthly_oversampled.parquet"
path_delta = ".data/tests/delta/df_agg_monthly_oversampled"

df = pd.read_parquet(path_pq)
df.head(5)

/Users/miloszhanczyk/Documents/uv/MOW_2/lab/sales_forecasting/.venv/lib/python3.10/site-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()
/Users/miloszhanczyk/Documents/uv/MOW_2/lab/sales_forecasting/.venv/lib/python3.10/site-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,date_block_num,shop_id,item_id,item_cnt_month,city_id,item_category_id,general_item_category_id,date_month,month_sin,month_cos,...,rolling_9,avg_shop_item_item_price_lag_1,avg_shop_item_item_cnt_day_lag_1,avg_item_item_price_lag_1,avg_item_item_cnt_day_lag_1,avg_shop_item_category_item_price_lag_1,avg_shop_item_category_item_cnt_day_lag_1,avg_item_category_item_price_lag_1,avg_item_category_item_cnt_day_lag_1,months_since_last_buy
0,20,0,0,0,0,40,11,8,0.866211,-0.500000,...,0.0,169.0,1.0,4488.0,1.0,270.50,1.163086,264.00,1.080078,0
1,21,0,0,0,0,40,11,9,0.707031,-0.707031,...,0.0,169.0,1.0,4488.0,1.0,263.50,1.150391,265.50,1.080078,1
2,22,0,0,0,0,40,11,10,0.500000,-0.866211,...,0.0,169.0,1.0,4488.0,1.0,263.75,1.170898,263.75,1.087891,2
3,23,0,0,0,0,40,11,11,0.258789,-0.965820,...,0.0,169.0,1.0,4488.0,1.0,262.25,1.138672,264.00,1.089844,3
4,24,0,0,0,0,40,11,0,0.000000,1.000000,...,0.0,169.0,1.0,4488.0,1.0,262.50,1.228516,262.00,1.131836,4


In [18]:
schema = pa.schema([
    ("date_block_num", pa.int32()),
    ("shop_id", pa.int32())    
])

In [19]:
dt = deltalake.DeltaTable.create(path_delta)

TypeError: DeltaTable.create() missing 1 required positional argument: 'schema'

In [21]:
deltalake.write_deltalake(path_delta, df[['date_block_num', 'shop_id']].head(10), mode='append')

In [22]:
dt = deltalake.DeltaTable(path_delta)


In [23]:
dt.schema()

Schema([Field(date_block_num, PrimitiveType("byte"), nullable=True), Field(shop_id, PrimitiveType("byte"), nullable=True), Field(__index_level_0__, PrimitiveType("long"), nullable=True)])

In [ ]:
deltalake.DeltaTable(path_delta).to_pandas()

In [ ]:
schema2 = schema.append(pa.field("item_id", pa.int32()))

In [ ]:
schema2

In [ ]:
deltalake.write_deltalake(path_delta, df[['date_block_num', 'shop_id']].iloc[10:15], schema=schema, mode='append')

In [ ]:
deltalake.DeltaTable(".data/tests/delta/df_agg_monthly_oversampled").to_pandas()

In [ ]:
deltalake.table.TableAlterer(dt).add_columns(fields=deltalake.Field(name="item_id", type="integer", nullable=False))

In [ ]:
deltalake.DeltaTable(path_delta).to_pandas()

In [ ]:
df[['date_block_num', 'shop_id', 'item_id']]

In [ ]:
deltalake.write_deltalake(path_delta, df[['date_block_num', 'shop_id', 'item_id']].iloc[15:20], mode='append')